In [1]:
# !pip install --upgrade pymongo certifi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 12.9 MB/s eta 0:00:00


In [2]:

# Install pymongo for MongoDB connection


# Import necessary libraries
from pymongo import MongoClient
import pprint

# Replace with your MongoDB Atlas connection string
connection_string = "mongodb+srv://vcn2fj:PW@cluster0.p2tka.mongodb.net/?retryWrites=true&w=majority&appName=Cluster0"

# Connect to MongoDB Atlas
client = MongoClient(connection_string)

# Access the sample_mflix database and the movies collection
db = client['sample_mflix']
collection = db['movies']


In [14]:
# Excercise 1
action_movie = collection.find_one({"genres": "Action"})
movies_after_2000 = collection.find({"year": {"$gt": 2000}}).limit(5)
high_rated_movies = collection.find({"imdb.rating": {"$gt": 8.5}}).limit(5)
action_adventure_movies = collection.find({"genres": {"$all": ["Action",
"Adventure"]}}).limit(5)
print("1. First movie with genre Action: "+action_movie['title'])
print("2. First five movies released after the year 2000:")
for movie in movies_after_2000:
    print("- " + movie['title'])
print("3. First five movies where IMDB rating greater than 8.5:")
for movie in high_rated_movies:
    print("- " + movie['title'])
print("4. First five movies where genre contains Action and Adventure:")
for movie in action_adventure_movies:
    print("- " + movie['title'])

1. First movie with genre Action: The Black Pirate
2. First five movies released after the year 2000:
- Kate & Leopold
- Glitter
- Kate & Leopold
- From Hell
- The Lord of the Rings: The Fellowship of the Ring
3. First five movies where IMDB rating greater than 8.5:
- City Lights
- Modern Times
- Casablanca
- Rear Window
- It's a Wonderful Life
4. First five movies where genre contains Action and Adventure:
- The Black Pirate
- Tarzan the Ape Man
- Beau Geste
- The Mark of Zorro
- The Adventures of Robin Hood


In [18]:
# Exercise 2
sorted_comedy_movies = collection.find({"genres": "Comedy"}).sort("imdb.rating", -1).limit(5)
sorted_drama_movies = collection.find({"genres": "Drama"}).sort("year", 1).limit(5)
print("1. First five movies where genre contains comedy (descending):")
for movie in sorted_comedy_movies:
    print("- " + movie['title'])
print("2. First five movies where genre contains drama (descending):")
for movie in sorted_drama_movies:
    print("- " + movie['title'])

1. First five movies where genre contains comedy (descending):
- The Chaos Class Failed the Class
- Heart of a Dog
- The Marathon Family
- Balkan Spy
- Who's Singin' Over There?
2. First five movies where genre contains drama (descending):
- A Corner in Wheat
- Traffic in Souls
- In the Land of the Head Hunters
- The Italian
- Regeneration


In [29]:
# Exercise 3
avg_rating_by_genre = collection.aggregate([
{"$unwind": "$genres"},
{"$group": {"_id": "$genres", "avg_rating": {"$avg": "$imdb.rating"}}},
{"$sort": {"avg_rating": -1}},
{"$limit": 5}
])
top_directors = collection.aggregate([
{"$group": {"_id": "$directors", "avg_rating": {"$avg": "$imdb.rating"}}},
{"$sort": {"avg_rating": -1}},
{"$limit": 5}
])
movies_per_year = collection.aggregate([
{"$group": {"_id": "$year", "total_movies": {"$sum": 1}}},
{"$sort": {"_id": 1}},
{"$limit": 5}
])

print("Average rating by genre:")
for genre in avg_rating_by_genre:
    print(f"Genre: {genre['_id']}, Average Rating: {genre['avg_rating']:.2f}")

print("\nTop directors by average rating:")
for director in top_directors:
    print(f"Director: {director['_id']}, Average Rating: {director['avg_rating']:.2f}")
print("")
print("Total movies per year (limited to 5 years):")
for year in movies_per_year:
    print(f"Year: {year['_id']}, Total Movies: {year['total_movies']}")

Average rating by genre:
Genre: Documentary, Average Rating: 7.50
Genre: Film-Noir, Average Rating: 7.40
Genre: Short, Average Rating: 7.38
Genre: War, Average Rating: 7.27
Genre: History, Average Rating: 7.26

Top directors by average rating:
Director: ['Michael Benson'], Average Rating: 9.00
Director: ['Slobodan Sijan'], Average Rating: 8.95
Director: ['Dusan Kovacevic', "Bozidar 'Bota' Nikolic"], Average Rating: 8.90
Director: ['Nesli èèlgeèen'], Average Rating: 8.80
Director: ['Rezo Chkheidze'], Average Rating: 8.75

Total movies per year (limited to 5 years):
Year: 1896, Total Movies: 2
Year: 1903, Total Movies: 1
Year: 1909, Total Movies: 1
Year: 1911, Total Movies: 2
Year: 1913, Total Movies: 1


In [30]:
# Exercise 4
collection.update_one({"title": "The Godfather"}, {"$set": {"imdb.rating": 9.5}})
collection.update_many({"genres": "Horror", "imdb.rating": {"$exists": False}}, {"$set":
{"imdb.rating": 6.0}})
collection.delete_many({"year": {"$lt": 1950}})

DeleteResult({'n': 639, 'electionId': ObjectId('7fffffff000000000000011e'), 'opTime': {'ts': Timestamp(1728061235, 656), 't': 286}, 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1728061235, 665), 'signature': {'hash': b'9?\xf9k3\xcf\xf0!\xde\xac9\xa4\xa3\xbd\xe7r\xb1\xe6?b', 'keyId': 7363326094432272385}}, 'operationTime': Timestamp(1728061235, 656)}, acknowledged=True)

In [32]:
# Exercise 5
# Create a text index on the title field
collection.create_index([("title", "text")])
# Find movies with 'love' in the title
love_movies = collection.find({"$text": {"$search": "love"}})
# Text search across title and plot, sorted by IMDb rating
collection.drop_index("title_text")
collection.create_index([("title", "text"), ("plot", "text")])
war_movies = collection.find({"$text": {"$search": "war"}}).sort("imdb.rating", -
1).limit(5)

In [33]:
# Exercise 6
action_high_rated_movies = collection.find({"genres": "Action", "imdb.rating": {"$gt":
8}}).sort("year", -1)
nolan_movies = collection.find({"directors": "Christopher Nolan", "imdb.rating": {"$gt":
8}}).sort("imdb.rating", -1).limit(3)